In [7]:
from datetime import datetime
import numpy as np
import bw2data as bd
from bw_temporalis import TemporalDistribution
from optimex import lca_processor

# Set the project (creates it if not existing)
bd.projects.set_current("__test_standalone_db__")

# BIOSPHERE
biosphere_data = {
    ("biosphere3", "CO2"): {
        "type": "emission",
        "name": "carbon dioxide",
        "CAS number": "000124-38-9"
    },
    ("biosphere3", "CH4"): {
        "type": "emission",
        "name": "methane, fossil",
        "CAS number": "000074-82-8"
    },
}
bd.Database("biosphere3").write(biosphere_data)

# BACKGROUND 2020
db_2020_data = {
    ("db_2020", "I1"): {
        "name": "node I1",
        "location": "somewhere",
        "reference product": "I1",
        "exchanges": [
            {"amount": 1, "type": "production", "input": ("db_2020", "I1")},
            {"amount": 1, "type": "biosphere", "input": ("biosphere3", "CO2")},
        ],
    },
    ("db_2020", "I2"): {
        "name": "node I2",
        "location": "somewhere",
        "reference product": "I2",
        "exchanges": [
            {"amount": 1, "type": "production", "input": ("db_2020", "I2")},
            {"amount": 1, "type": "biosphere", "input": ("biosphere3", "CH4")},
        ],
    },
}
bd.Database("db_2020").write(db_2020_data)

# BACKGROUND 2030
db_2030_data = {
    ("db_2030", "I1"): {
        "name": "node I1",
        "location": "somewhere",
        "reference product": "I1",
        "exchanges": [
            {"amount": 1, "type": "production", "input": ("db_2030", "I1")},
            {"amount": 0.9, "type": "biosphere", "input": ("biosphere3", "CO2")},
        ],
    },
    ("db_2030", "I2"): {
        "name": "node I2",
        "location": "somewhere",
        "reference product": "I2",
        "exchanges": [
            {"amount": 1, "type": "production", "input": ("db_2030", "I2")},
            {"amount": 0.9, "type": "biosphere", "input": ("biosphere3", "CH4")},
        ],
    },
}
bd.Database("db_2030").write(db_2030_data)

# FOREGROUND - temporally distributed
foreground_data = {
    ("foreground", "P1"): {
        "name": "process P1",
        "location": "somewhere",
        "reference product": "F1",
        "exchanges": [
            {
                "amount": 1,
                "type": "production",
                "input": ("foreground", "P1"),
                "temporal_distribution": TemporalDistribution(
                    date=np.array(range(4), dtype="timedelta64[Y]"),
                    amount=np.array([0, 0.5, 0.5, 0]),
                ),
            },
            {
                "amount": 27.5,
                "type": "technosphere",
                "input": ("db_2020", "I1"),
                "temporal_distribution": TemporalDistribution(
                    date=np.array(range(4), dtype="timedelta64[Y]"),
                    amount=np.array([1, 0, 0, 0]),
                ),
            },
            {
                "amount": 20,
                "type": "biosphere",
                "input": ("biosphere3", "CO2"),
                "temporal_distribution": TemporalDistribution(
                    date=np.array(range(4), dtype="timedelta64[Y]"),
                    amount=np.array([0, 0.5, 0.5, 0]),
                ),
            },
        ],
    },
    ("foreground", "P2"): {
        "name": "process P2",
        "location": "somewhere",
        "reference product": "F1",
        "exchanges": [
            {
                "amount": 1,
                "type": "production",
                "input": ("foreground", "P2"),
                "temporal_distribution": TemporalDistribution(
                    date=np.array(range(4), dtype="timedelta64[Y]"),
                    amount=np.array([0, 0.5, 0.5, 0]),
                ),
            },
            {
                "amount": 1,
                "type": "technosphere",
                "input": ("db_2020", "I2"),
                "temporal_distribution": TemporalDistribution(
                    date=np.array(range(4), dtype="timedelta64[Y]"),
                    amount=np.array([0, 0, 0, 1]),
                ),
            },
            {
                "amount": 20,
                "type": "biosphere",
                "input": ("biosphere3", "CO2"),
                "temporal_distribution": TemporalDistribution(
                    date=np.array(range(4), dtype="timedelta64[Y]"),
                    amount=np.array([0, 0.5, 0.5, 0]),
                ),
            },
        ],
    },
}
bd.Database("foreground").write(foreground_data)


bd.Method(("GWP", "example")).write([
    (("biosphere3", "CO2"), 1),
    (("biosphere3", "CH4"), 27),
])

bd.Method(("land use", "example")).write([
    (("biosphere3", "CO2"), 2),
    (("biosphere3", "CH4"), 1),
])

100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 


In [8]:

# For optimex to work, we need to manually define which processes  produce desired 
# products labeled as functional flows
foreground = bd.Database("foreground")
for act in foreground:
    act["functional flow"] = "F1"
    act.save()

# Define temporally distirbuted demand from 2020 to 2030
td_demand = TemporalDistribution(
    date=np.arange(2020 - 1970, 10, dtype="datetime64[Y]"),
    amount=np.asarray([0, 0, 10, 5, 10, 5, 10, 5, 10, 5]),
)


lca_data_processor = lca_processor.LCADataProcessor(
    demand={"F1": td_demand},
    start_date=datetime.strptime("2020", "%Y"),
    methods={
        "climate_change" : ("GWP", "example"),
        "land_use" : ("land use", "example"),
    },
    database_date_dict={
        "db_2020": datetime.strptime("2020", "%Y"),
        "db_2030": datetime.strptime("2030", "%Y"),
        "foreground": "dynamic",
    },
    temporal_resolution="year",
    timehorizon=100,
)

In [13]:
lca_data_processor.parse_demand()
lca_data_processor.construct_foreground_tensors()
lca_data_processor.set_process_operation_time({"P1": (1,2), "P2": (1,2)})
lca_data_processor.sequential_inventory_tensor_calculation()
lca_data_processor.construct_mapping_matrix()
lca_data_processor.construct_characterization_matrix(base_method_name="climate_change", dynamic=True, metric="CRF");
lca_data_processor.construct_characterization_matrix(base_method_name="land_use", dynamic=False)
lca_data_processor.characterization

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00, 27.16it/s]
2025-05-20 17:46:52.861 | INFO     | dynamic_characterization.dynamic_characterization:characterize:82 - No custom dynamic characterization functions provided. Using default dynamic             characterization functions. The flows that are characterized are based on the selection                of the initially chosen impact category.
C:\Users\HP\Desktop\Repos\optimex-1\optimex\lca_processor.py:743: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  df_characterized["date"].dt.to_pydatetime()
2025-05-20 17:46:53.126 | INFO     | dynamic_characterization.dynamic_characterization:characterize:82 - No custom dynamic characterization functions provided. Using default dynamic             characterization functions. The flows that are characterized are base

{('climate_change', 'CO2', 2020): 8.856378067710995e-14,
 ('climate_change', 'CO2', 2021): 8.78632948322376e-14,
 ('climate_change', 'CO2', 2022): 8.716115201983699e-14,
 ('climate_change', 'CO2', 2023): 8.645732530491629e-14,
 ('climate_change', 'CO2', 2024): 8.575178705349817e-14,
 ('climate_change', 'CO2', 2025): 8.50445089133486e-14,
 ('climate_change', 'CO2', 2026): 8.433546179417147e-14,
 ('climate_change', 'CO2', 2027): 8.362461584725384e-14,
 ('climate_change', 'CO2', 2028): 8.291194044454685e-14,
 ('climate_change', 'CO2', 2029): 8.219740415716608e-14,
 ('climate_change', 'CH4', 2020): 2.3651673669270527e-12,
 ('climate_change', 'CH4', 2021): 2.3651198384711042e-12,
 ('climate_change', 'CH4', 2022): 2.3650681065838066e-12,
 ('climate_change', 'CH4', 2023): 2.36501179951239e-12,
 ('climate_change', 'CH4', 2024): 2.3649505126261558e-12,
 ('climate_change', 'CH4', 2025): 2.3648838055087402e-12,
 ('climate_change', 'CH4', 2026): 2.364811198793221e-12,
 ('climate_change', 'CH4', 20

In [10]:
lca_data_processor.construct_characterization_matrix(base_method_name="land_use", dynamic=False);

In [11]:
from optimex import converter

con = converter.Converter(lca_data_processor)
model_inputs = con.combine_and_check(category_impact_limit = {"land_use":1e5})

In [12]:
from optimex import optimizer

model = optimizer.create_model(
    inputs=model_inputs,
    objective_category="climate_change",
    name=f"abstract_system_model_flex",
    flexible_operation=True,
)
model, results = optimizer.solve_model(model, solver_name="gurobi")


Read LP format model from file C:\Users\HP\AppData\Local\Temp\tmpg36rhpu4.pyomo.lp
Reading time = 0.01 seconds
x1: 53 rows, 40 columns, 151 nonzeros
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8365U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 53 rows, 40 columns and 151 nonzeros
Model fingerprint: 0x9c024550
Coefficient statistics:
  Matrix range     [5e-01, 6e+01]
  Objective range  [9e-01, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 1e+05]
Presolve removed 40 rows and 20 columns
Presolve time: 0.00s
Presolved: 13 rows, 20 columns, 50 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4000000e+03   1.125000e+01   0.000000e+00      0s
       9    2.4564000e+03   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.01 seconds (0.00 work units)